In [51]:
# System imports
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
from time import process_time

# Extra library imports
from PIL import Image
from scipy.optimize import minimize
import scipy.ndimage as ndi
import pandas as pd
import cv2
from skimage.draw import line
from skimage.measure import label, regionprops
import mahotas as mh
from scipy.stats import kurtosis
from scipy.stats import skew
import glob

# **SYMMETRY INDEX**

In [52]:
def symmetry_index(mask_arr):
    # Flip the mask around the y-axis
    flipped_mask = np.fliplr(mask_arr)
    # Calculate the absolute difference between the original and flipped mask
    diff = np.abs(mask_arr - flipped_mask)

    # Calculate the symmetry index
    symmetry_index = np.sum(diff) / np.sum(mask_arr)

    return symmetry_index

# **RATIO TWO LONGEST ORTHO LINES**

In [53]:
def Longest_ortho_lines_ratio(mask_arr,im_arr):
    return 0

# **RATIO NUM_PIX_BUG / NUM_PIX_IM**

In [54]:
def number_of_pixel_ratio(mask_arr, im_arr):
    # Assume mask_arr is your mask array and image is your image array

    # Calculate the number of pixels of the bug
    num_pixels_bug = np.sum(mask_arr > 0)
    # Calculate the number of pixels in the full image
    num_pixels_image = im_arr.size
    # Calculate the ratio
    ratio = num_pixels_bug / num_pixels_image

    return ratio

# **MIN,MAX,MEAN,MEDIAN,STANDARD DEVIATION RGB MASK**

In [55]:
def rgb_mask_features(mask_arr, im_arr):
    # Assuming the bug mask is stored in the variable 'mask_arr' as a numpy array

    # Get the RGB values within the bug mask
    bug_pixels = im_arr[mask_arr == 1]
    # Get the Red, Green, and Blue channels
    red_channel = bug_pixels[:, 0]
    green_channel = bug_pixels[:, 1]
    blue_channel = bug_pixels[:, 2]

    # Calculate the minimum, maximum, and mean values for each channel
    red_min = np.min(red_channel)
    red_max = np.max(red_channel)
    red_mean = np.mean(red_channel)

    green_min = np.min(green_channel)
    green_max = np.max(green_channel)
    green_mean = np.mean(green_channel)

    blue_min = np.min(blue_channel)
    blue_max = np.max(blue_channel)
    blue_mean = np.mean(blue_channel)

    # Calculate median and standard deviation for red channel
    red_median = np.median(red_channel)
    red_std = np.std(red_channel)

    # Calculate median and standard deviation for green channel
    green_median = np.median(green_channel)
    green_std = np.std(green_channel)

    # Calculate median and standard deviation for blue channel
    blue_median = np.median(blue_channel)
    blue_std = np.std(blue_channel)

    red_features = [red_min, red_max, red_mean, red_median, red_std]
    green_features = [green_min, green_max, green_mean, green_median, green_std]
    blue_features = [blue_min, blue_max, blue_mean, blue_median, blue_std]

    return (red_features,blue_features,green_features)

# **HARALICK TEXTURE FEATURE**

In [56]:
def haralick_features(mask_arr, im_arr):
    # Assuming the bug mask is stored in the variable 'mask_arr' as a numpy array

    # Convert the image to grayscale
    gray_image = mh.colors.rgb2gray(im_arr*mask_arr[:,:,None])
    gray_image = gray_image.astype(np.uint8)

    # Calculate Haralick texture features
    haralick_features = mh.features.haralick(gray_image, return_mean=True)

    return haralick_features

# **ECCENTRICITY FEATURE**

In [57]:
def eccentricity(mask_arr):
    # Calculate the eccentricity
    props = regionprops(mask_arr)
    eccentricity = props[0].eccentricity

    return eccentricity

# **KURTHOSIS,SKEWNESS FEATURE**

In [58]:
def kurthosis_skewness(mask_arr, im_arr):
    # Assuming im_arr is your image array and mask_arr is your mask
    # Apply the mask to the image
    masked_im = im_arr * mask_arr[:,:,None]

    # Flatten the array to 1D for the calculation
    flattened_im = masked_im.flatten()

    # Calculate kurtosis
    kurt = kurtosis(flattened_im)

    # Assuming flattened_im is your flattened image array from the previous step
    # Calculate skewness
    skewness = skew(flattened_im)

    return (kurt,skewness) 

# **Creating all features in a dataset**

In [59]:
def Creating_all_features(mask_arr, im_arr):
    # Calculate the symmetry index
    symmetry = symmetry_index(mask_arr)

    longest_ratio = Longest_ortho_lines_ratio(mask_arr,im_arr)
    # Calculate the number of pixels ratio
    num_pixels_ratio = number_of_pixel_ratio(mask_arr, im_arr)

    # Calculate the RGB features
    rgb_features = rgb_mask_features(mask_arr, im_arr)

    # Calculate the Haralick features
    haralick = haralick_features(mask_arr, im_arr)

    # Calculate the eccentricity
    ecc = eccentricity(mask_arr)

    # Calculate the kurtosis and skewness
    kurt, skewness = kurthosis_skewness(mask_arr, im_arr)

    return (symmetry, longest_ratio, num_pixels_ratio, rgb_features, haralick, ecc, kurt, skewness)

In [75]:
# Get the file paths of the images and masks in the train folder
image_paths = [f'../train/{i}.jpg' for i in range(1, 3)]
mask_paths = [f'../train/masks/binary_{i}.tif' for i in range(1, 3)]


dataframe = pd.DataFrame(columns=['symmetry', 'longest_ratio', 'num_pixels_ratio', 'rgb_features', 'haralick', 'ecc', 'kurt', 'skewness'])

# Iterate over the image and mask paths
for i, (image_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
    print(image_path, mask_path)
    im_arr = np.array(Image.open(image_path))
    mask_arr = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask_arr = np.where(mask_arr > 0, 1, 0)

    # Appeler la fonction Creating_all_features pour obtenir les valeurs
    features = Creating_all_features(mask_arr, im_arr)

    # Create a new DataFrame from the features and append it to df
    dataframe.loc[i] = features


../train/1.jpg ../train/masks/binary_1.tif
../train/2.jpg ../train/masks/binary_2.tif


In [76]:
dataframe

,symmetry,longest_ratio,num_pixels_ratio,rgb_features,haralick,ecc,kurt,skewness
0,0.695058,0,0.002476,"([5, 208, 68.08574489687165, 56.0, 47.95539885...","[0.9848405154861913, 4.633900391026453, 0.9401...",0.392019,415.810811,19.208069
1,0.833665,0,0.002851,"([2, 248, 63.786497914961615, 55.0, 42.1535075...","[0.9825862190349237, 5.347013532380983, 0.9311...",0.595904,330.090332,17.303837
